Task 3.1

You are offered a food demand table that includes sushi, pizza, and burgers, as well as their corresponding prices and quality ratings. The task is to find the most informative criterion for the first split into groups.

In [1]:
import pandas as pd

data = [['Pizza',	1,	2, 'No'],
['Pizza',	3, 2,	'Yes'],
['Pizza',	5,	1,	'No'],
['Pizza',	1,	3,	'Yes'],
['Pizza',	2, 3,	'No'],
['Sushi',	3,	2,	'Yes'],
['Sushi',	1,	5,	'Yes'],
['Sushi',	3,	4,	'No'],
['Sushi',	5,	2,	'No'],
['Sushi',	5,	5,	'Yes'],
['Sushi',	3,	5,	'No'],
['Sushi',	5,	5,	'No'],
['Sushi',	4,	4,	'Yes'],
['Burgers',	4,	4,	'No'],
['Burgers',	3,	3,	'No'],
['Burgers',	2,	2,	'No'],
['Burgers',	3,	3,	'Yes'],
['Burgers',	2,	2,	'Yes'],
['Burgers',	1,	1,	'Yes'],
['Burgers',	1,	1,	'No']]

df = pd.DataFrame(data, columns=['Food','Price', 'Taste', 'Demand'])
df

,Food,Price,Taste,Demand
0,Pizza,1,2,No
1,Pizza,3,2,Yes
2,Pizza,5,1,No
3,Pizza,1,3,Yes
4,Pizza,2,3,No
5,Sushi,3,2,Yes
6,Sushi,1,5,Yes
7,Sushi,3,4,No
8,Sushi,5,2,No
9,Sushi,5,5,Yes


Calculate the initial entropy of the response H(Demand).

In [2]:
from collections import Counter
cnt = Counter(df['Demand'])
cnt

Counter({'No': 11, 'Yes': 9})

In [3]:
import math
total = sum(cnt.values())
P_demand = [val/total for val in cnt.values()]
H_demand = sum([-val/total*math.log(val/total, 2) for val in cnt.values()])
print('p =', P_demand, '\th =', H_demand)

p = [0.55, 0.45] 	h = 0.9927744539878084


In [4]:
#since the highest entropy of the experiment can be equal to 1, we can conclude there is only uncertainty.

Fill out the probability table of outcomes of the experiment.

P(Demand = 'YES' | Food = 'Pizza'), P(Demand = 'NO' | Food = 'Pizza') 

P(Demand = 'YES' | Food = 'Sushi'), P(Demand = 'NO' | Food = 'Sushi') 

P(Demand = 'YES' | Food = 'Burgers'), P(Demand = 'NO' | Food = 'Burgers') 

In [5]:
#P(A|B) = P(A and B)/P(B) = num(A and B)/num(B)
def conditional_p(col_name1, value1, col_name2, value2, debug = 'no'):
  numAB = df.loc[df[col_name1] == value1].loc[df[col_name2] == value2].shape[0]
  numB = df.loc[df[col_name2] == value2].shape[0]
  if debug == 'yes':
    print(f'{numAB}/{numB}={numAB/numB}')
  return numAB/numB

def classic_p(col_name, value):
  return df.loc[df[col_name] == value].shape[0]/df.shape[0]

In [6]:
for demand_type in set(df['Demand']):
  for food_type in set(df['Food']):
    print('P(Demand=', demand_type,'| Food=', food_type, ')=', conditional_p('Demand', demand_type, 'Food', food_type, debug='yes'))

2/5=0.4
P(Demand= Yes | Food= Pizza )= 0.4
4/8=0.5
P(Demand= Yes | Food= Sushi )= 0.5
3/7=0.42857142857142855
P(Demand= Yes | Food= Burgers )= 0.42857142857142855
3/5=0.6
P(Demand= No | Food= Pizza )= 0.6
4/8=0.5
P(Demand= No | Food= Sushi )= 0.5
4/7=0.5714285714285714
P(Demand= No | Food= Burgers )= 0.5714285714285714


Calculate the conditional entropies of the experiment (Food, Demand) given that sushi, pizza, or burger is chosen.

Enter the value of conditional entropy given that burger is chosen H(Demand|Burgers).

In [7]:
#H(Demand|food_type)  = SUM [ -P(dem_i|food_type) log P(dem_i|food_type) ]
H_dem_bur = sum([-conditional_p('Demand', dem, 'Food', 'Burgers')*math.log(conditional_p('Demand', dem, 'Food', 'Burgers'), 2) for dem in set(df['Demand'])])
H_dem_bur

0.9852281360342516

In [8]:
H_dem_piz = sum([-conditional_p('Demand', dem, 'Food', 'Pizza')*math.log(conditional_p('Demand', dem, 'Food', 'Pizza'), 2) for dem in set(df['Demand'])])
H_dem_piz

0.9709505944546686

In [9]:
H_dem_sus = sum([-conditional_p('Demand', dem, 'Food', 'Sushi')*math.log(conditional_p('Demand', dem, 'Food', 'Sushi'), 2) for dem in set(df['Demand'])])
H_dem_sus

1.0

Calculate the information gain (Demand|Food)

In [10]:
#IG(Demand|Food) = H(Demand) - H(Demand|Food)
#H(Demand|Food) = E[H(Demand|Food_i)]
H_dem_food = classic_p('Food', 'Burgers') * H_dem_bur + classic_p('Food', 'Pizza') * H_dem_piz + classic_p('Food', 'Sushi') * H_dem_sus
IG_dem_food = H_demand - H_dem_food
IG_dem_food

0.005206957762153186

Let's now create a function to calculate IG from the beginning 

In [11]:
#calculate IG(arg1|arg2) in dataframe df
def get_ig(arg1, arg2, data_frame, log_base = 2):
  cnt = Counter(df[arg1])
  total = sum(cnt.values())
  P_a1 = [val/total for val in cnt.values()]
  H_a1 = sum([-val/total * math.log(val/total, log_base) for val in cnt.values()])

  H_a1_a2_arr = []
  for a2_type in set(df[arg2]):
    H_a1_a2_arr.append(classic_p(arg2, a2_type) * sum([-conditional_p(arg1, a1, arg2, a2_type)*math.log(conditional_p(arg1, a1, arg2, a2_type), log_base) for a1 in set(df[arg1])]))

  ig = H_a1 - sum(H_a1_a2_arr)
  return ig


In [12]:
#check the result for value we know
get_ig('Demand', 'Food', df)

0.005206957762153297

Calculate the information gain (Demand|Price)

In [13]:
get_ig('Demand', 'Price', df)

0.05003680537414135

In [14]:
get_ig('Demand', 'Taste', df)

0.017285703771461436